**Install conda, taming-transformers and its dependencies**

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

!git clone https://github.com/rbbb/taming-transformers
%cd /content/taming-transformers
!git checkout stable-diff-vae-finetuning

!conda env update -n taming-transformers -f environment.yaml
!conda run -n taming-transformers pip install -e .

**Put some images in /content/src_img_here/**

In [ ]:
!mkdir /content/src_img_here/
#directory /0000/ from danbooru 2020 data containing 3.5k images
!unzip /content/drive/MyDrive/danbooru_tiny.zip -d /content/src_img_here/

**Find all images, put their name in train_img.txt and val_img.txt (10 validation images)**

In [ ]:
import glob
img_filenames = list(glob.glob("/content/src_img_here/**/*.*", recursive=True))
img_filenames = [i for i in img_filenames if (i.endswith(".png") or i.endswith(".jpg") or i.endswith(".jpeg"))]
with open("/content/taming-transformers/train_img.txt","w") as f:
  f.write("\n".join(img_filenames[10:]))
with open("/content/taming-transformers/val_img.txt","w") as f:
  f.write("\n".join(img_filenames[0:10]))


**Get a VAE from civitai, convert from safetensors to ckpt**

In [ ]:
!mkdir /content/src_vae_here/
!wget -O /content/src_vae_here/vae.safetensors https://civitai.com/api/download/models/88156 #Clear VAE

!pip install safetensors torch
from safetensors import safe_open
import torch
vae = {}
with safe_open("/content/src_vae_here/vae.safetensors", framework="pt", device=0) as f:
    vae = {k: f.get_tensor(k) for k in f.keys()}
torch.save({'state_dict':vae}, "/content/src_vae_here/vae.pt")

**Change the parameters in the /content/taming-transformers/configs/finetune_vae.yaml (name of VAE file and batch size)**

In [ ]:
import yaml
with open('/content/taming-transformers/configs/finetune_vae.yaml','r') as file:
    conf = yaml.load(file, Loader=yaml.FullLoader)

conf['model']['params']['ckpt_path'] = "\"/content/src_vae_here/vae.pt\""
conf['data']['params']['batch_size'] = "5"

with open('/content/taming-transformers/configs/finetune_vae.yaml','w') as file:
    yaml.dump(conf, file)



**Run**

In [ ]:
%cd /content/taming-transformers
!conda run --no-capture-output -n taming-transformers python main.py --base configs/finetune_vae.yaml -t True --gpus 0, --accumulate_grad_batches 6